## Install the necessary packages

In [1]:
# ALERT: ends up this package produces WAY too many emails - FRONT only allows 100 emails per day
!pip install email_permutations

# Let's try with tldextract to take website links into host name
!pip install tldextract

In [2]:
import pandas as pd

In [3]:
from tldextract import extract

In [8]:
df = pd.read_csv('LinkedinScraper/recruiters.csv')

In [9]:
# take only the first 3 columns of df (i don't want email from r3sourcing just to test)
df = df[['Name', 'Domain Name']]
df.head()

,Name,Domain Name
0,Molly Martin,metacareers.com
1,Annette Erdozaincy,metacareers.com
2,Benjamin C.,metacareers.com
3,Kim Grounds,metacareers.com
4,Don Lee,metacareers.com


In [10]:
# cleans our website link
def getHostName(website):
    tsd, td, tsu = extract(website)
    url = td + '.' + tsu
    return url

In [11]:
# splits full name of contact into first name and last name
# fills in first name with full name if last name is not present or difficult to process
# TODO(Chris): add logic for middle names: https://moonbooks.org/Articles/How-to-split-a-full-name-column-into-a-first-and-last-name-columns-with-pandas-in-python-/
df[['First Name', 'Last Name']] = df['Name'].loc[df['Name'].str.split().str.len() == 2].str.split(expand=True)
df['Domain Name'] = df['Domain Name'].apply(lambda x: getHostName(x))
df['First Name'].fillna(df['Name'],inplace=True)
df['First Name'].fillna("",inplace=True)
df['Last Name'].fillna("",inplace=True)
df.head()

,Name,Domain Name,First Name,Last Name
0,Molly Martin,metacareers.com,Molly,Martin
1,Annette Erdozaincy,metacareers.com,Annette,Erdozaincy
2,Benjamin C.,metacareers.com,Benjamin,C.
3,Kim Grounds,metacareers.com,Kim,Grounds
4,Don Lee,metacareers.com,Don,Lee


In [12]:
# clean our first name, last name. and company columns
df['First Name'] = df['First Name'].str.strip()
df['First Name'] = df['First Name'].str.lower()
df['First Name'] = df['First Name'].str.replace('.','')
df['Last Name'] = df['Last Name'].str.lower()
df['Last Name'] = df['Last Name'].str.strip()
df['Last Name'] = df['Last Name'].str.replace('.','')
# df['Company Name'] = df['Company Name'].str.lower()
# df['Company Name'] = df['Company Name'].str.replace(' ', '')

# drop empty last name rows
df = df[df['Last Name'] != '']
#drop empty first name rows
df = df[df['First Name'] != '']
df.head()

<ipython-input-12-a6127b8e140f>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['First Name'] = df['First Name'].str.replace('.','')
<ipython-input-12-a6127b8e140f>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Last Name'] = df['Last Name'].str.replace('.','')


,Name,Domain Name,First Name,Last Name
0,Molly Martin,metacareers.com,molly,martin
1,Annette Erdozaincy,metacareers.com,annette,erdozaincy
2,Benjamin C.,metacareers.com,benjamin,c
3,Kim Grounds,metacareers.com,kim,grounds
4,Don Lee,metacareers.com,don,lee


In [15]:
#test permuted emails package
def nameToEmail(first_name, last_name, domain_name):
	listOfPossibleEndings = ['.com', '.net', '.io'] # TODO(chris): map these out as well??
	# domain_name = company_name + '.com'

	# 3 of the most popular sequences:
	# {first}.{last}
	# {f}{last}
	# {first}{last}
	permuted_emails = [
		first_name + '.' + last_name + '@' + domain_name,
		first_name + last_name + '@' + domain_name,
		first_name[0] + last_name + '@' + domain_name,
	]

	# join permuted_emails together into string separated by comma
	permuted_emails = ','.join(permuted_emails)
	return permuted_emails

In [16]:
# map the nameToEmail function across the dataframe df
for i in df.index:
    df.loc[i, 'Email'] = nameToEmail(df.loc[i, 'First Name'], df.loc[i, 'Last Name'], df.loc[i, 'Domain Name'])
df.head()

,Name,Domain Name,First Name,Last Name,Email
0,Molly Martin,metacareers.com,molly,martin,"molly.martin@metacareers.com,mollymartin@metac..."
1,Annette Erdozaincy,metacareers.com,annette,erdozaincy,"annette.erdozaincy@metacareers.com,annetteerdo..."
2,Benjamin C.,metacareers.com,benjamin,c,"benjamin.c@metacareers.com,benjaminc@metacaree..."
3,Kim Grounds,metacareers.com,kim,grounds,"kim.grounds@metacareers.com,kimgrounds@metacar..."
4,Don Lee,metacareers.com,don,lee,"don.lee@metacareers.com,donlee@metacareers.com..."


In [18]:
df = df.set_index(['Name', 'Domain Name', 'First Name', 'Last Name']).apply(lambda x: x.str.split(',').explode()).reset_index()
df.reset_index(drop=True, inplace=True)

In [19]:
df.to_csv('out/permutedEmailsResult.csv')